In [321]:
## Imports
import os
from pathlib import Path
import numpy as np
import pandas as pd
from typing import Tuple, Dict

import polars as pl

import warnings
warnings.filterwarnings('ignore')

In [322]:
## Configuration and Data Loading (kaggle_evaluation only)
# import kaggle_evaluation.default_inference_server as kdeval
# DATA_DIR = Path('/kaggle/input/hull-tactical-market-prediction')

## Configuration and Data Loading (local version only)
DATA_DIR = Path("01_data")

# Read CSV files from data_path
TRAIN_PATH = DATA_DIR / 'train.csv'
TEST_PATH  = DATA_DIR / 'test.csv'

VALIDATION_SIZE = 2700          # days, approx. 30% of data
# RANDOM_SEED = 42

import random
RANDOM_SEED = random.randint(1, 10000)

VOL_MULTIPLIER_LIMIT = 1.2
VOL_WINDOW = 20

def time_split_train_val(df: pd.DataFrame, val_size: int = 2700):
    df = df.sort_values('date_id').reset_index(drop=True)
    train_df = df.iloc[:-val_size].copy()
    val_df   = df.iloc[-val_size:].copy()
    return train_df, val_df

train_raw = pd.read_csv(TRAIN_PATH)
test_raw  = pd.read_csv(TEST_PATH)

# lower column names
train_raw.columns = [c.lower() for c in train_raw.columns]
test_raw.columns  = [c.lower() for c in test_raw.columns]
train_raw.shape, test_raw.shape

((8990, 98), (10, 99))

In [323]:
# main_features = ['m4', 'm11', 'v13', 's2', 'p6', 'm12', 's1', 'v9', 'e17', 'p7'] # from EDA

# # keep only main features in train_raw
# train_p = train_raw[['date_id', 'forward_returns', 'risk_free_rate', 'market_forward_excess_returns'] + main_features].copy()
# test_p  = test_raw[['date_id'] + main_features].copy()

In [324]:
# train_p.shape

In [325]:
# train_p.head()

In [326]:
# # check for nan in train_p
# train_p.isna().sum()

In [327]:
## Feature Preparation
excluded = {'date_id', 'forward_returns', 'risk_free_rate', 'market_forward_excess_returns'}
feature_cols = [c for c in train_raw.columns if c not in excluded]
feature_cols = [c for c in feature_cols if c in test_raw.columns]

In [328]:
feature_cols

['d1',
 'd2',
 'd3',
 'd4',
 'd5',
 'd6',
 'd7',
 'd8',
 'd9',
 'e1',
 'e10',
 'e11',
 'e12',
 'e13',
 'e14',
 'e15',
 'e16',
 'e17',
 'e18',
 'e19',
 'e2',
 'e20',
 'e3',
 'e4',
 'e5',
 'e6',
 'e7',
 'e8',
 'e9',
 'i1',
 'i2',
 'i3',
 'i4',
 'i5',
 'i6',
 'i7',
 'i8',
 'i9',
 'm1',
 'm10',
 'm11',
 'm12',
 'm13',
 'm14',
 'm15',
 'm16',
 'm17',
 'm18',
 'm2',
 'm3',
 'm4',
 'm5',
 'm6',
 'm7',
 'm8',
 'm9',
 'p1',
 'p10',
 'p11',
 'p12',
 'p13',
 'p2',
 'p3',
 'p4',
 'p5',
 'p6',
 'p7',
 'p8',
 'p9',
 's1',
 's10',
 's11',
 's12',
 's2',
 's3',
 's4',
 's5',
 's6',
 's7',
 's8',
 's9',
 'v1',
 'v10',
 'v11',
 'v12',
 'v13',
 'v2',
 'v3',
 'v4',
 'v5',
 'v6',
 'v7',
 'v8',
 'v9']

In [329]:
def prepare_df(df: pd.DataFrame, median_map: Dict[str, float], feature_cols: list) -> pd.DataFrame:
    df = df.copy()
    for c in feature_cols:
        # if c not in df.columns:
        #     df[c] = 0.0
        #     df[f'{c}_was_na'] = 1
        #     continue
        if df[c].dtype.kind in 'fiu':
            med = median_map.get(c, 0.0)
            was_na = df[c].isna().astype(int)
            df[c] = df[c].fillna(med)
            df[f'{c}_was_na'] = was_na
        else:
            df[c] = pd.to_numeric(df[c], errors='coerce')
            med = median_map.get(c, 0.0)
            was_na = df[c].isna().astype(int)
            df[c] = df[c].fillna(med)
            df[f'{c}_was_na'] = was_na
    return df

In [330]:
## Train / Validation Split and Median Imputation
train_df, val_df = time_split_train_val(train_raw, val_size=VALIDATION_SIZE)

median_map = {c: float(train_df[c].median(skipna=True)) if train_df[c].dtype.kind in 'fiu' else 0.0 
              for c in feature_cols}

train_p = prepare_df(train_df, median_map, feature_cols)
val_p   = prepare_df(val_df, median_map, feature_cols)
test_p  = prepare_df(test_raw, median_map, feature_cols)

final_features = [f for c in feature_cols for f in (c, f"{c}_was_na")]
print("Number of features:", len(final_features))

Number of features: 188


In [331]:
# train_p.head()

In [332]:
print(train_p.columns)

Index(['date_id', 'd1', 'd2', 'd3', 'd4', 'd5', 'd6', 'd7', 'd8', 'd9',
       ...
       'v12_was_na', 'v13_was_na', 'v2_was_na', 'v3_was_na', 'v4_was_na',
       'v5_was_na', 'v6_was_na', 'v7_was_na', 'v8_was_na', 'v9_was_na'],
      dtype='object', length=192)


In [333]:
final_features

['d1',
 'd1_was_na',
 'd2',
 'd2_was_na',
 'd3',
 'd3_was_na',
 'd4',
 'd4_was_na',
 'd5',
 'd5_was_na',
 'd6',
 'd6_was_na',
 'd7',
 'd7_was_na',
 'd8',
 'd8_was_na',
 'd9',
 'd9_was_na',
 'e1',
 'e1_was_na',
 'e10',
 'e10_was_na',
 'e11',
 'e11_was_na',
 'e12',
 'e12_was_na',
 'e13',
 'e13_was_na',
 'e14',
 'e14_was_na',
 'e15',
 'e15_was_na',
 'e16',
 'e16_was_na',
 'e17',
 'e17_was_na',
 'e18',
 'e18_was_na',
 'e19',
 'e19_was_na',
 'e2',
 'e2_was_na',
 'e20',
 'e20_was_na',
 'e3',
 'e3_was_na',
 'e4',
 'e4_was_na',
 'e5',
 'e5_was_na',
 'e6',
 'e6_was_na',
 'e7',
 'e7_was_na',
 'e8',
 'e8_was_na',
 'e9',
 'e9_was_na',
 'i1',
 'i1_was_na',
 'i2',
 'i2_was_na',
 'i3',
 'i3_was_na',
 'i4',
 'i4_was_na',
 'i5',
 'i5_was_na',
 'i6',
 'i6_was_na',
 'i7',
 'i7_was_na',
 'i8',
 'i8_was_na',
 'i9',
 'i9_was_na',
 'm1',
 'm1_was_na',
 'm10',
 'm10_was_na',
 'm11',
 'm11_was_na',
 'm12',
 'm12_was_na',
 'm13',
 'm13_was_na',
 'm14',
 'm14_was_na',
 'm15',
 'm15_was_na',
 'm16',
 'm16_was_na'

In [334]:
# === Bayesian Ridge Training ===
from sklearn.linear_model import BayesianRidge
from sklearn.metrics import mean_squared_error

# Prepara i dati (usando gli stessi DataFrame)
X_train = train_p[final_features]
y_train = train_p['forward_returns']
X_val   = val_p[final_features]
y_val   = val_p['forward_returns']

# Check for NaN values and handle them
print(f"NaN values in X_train: {X_train.isnull().sum().sum()}")
print(f"NaN values in y_train: {y_train.isnull().sum()}")

# Fill any remaining NaN values
X_train = X_train.fillna(0)
y_train = y_train.fillna(0)
X_val = X_val.fillna(0)
y_val = y_val.fillna(0)

# count nan in X_train, y_train, X_val, y_val
print(f"After filling NaN values:")
print(f"NaN values in X_train: {X_train.isnull().sum().sum()}")
print(f"NaN values in y_train: {y_train.isnull().sum()}")
print(f"NaN values in X_val: {X_val.isnull().sum().sum()}")
print(f"NaN values in y_val: {y_val.isnull().sum()}")

NaN values in X_train: 6290
NaN values in y_train: 0
After filling NaN values:
NaN values in X_train: 0
NaN values in y_train: 0
NaN values in X_val: 0
NaN values in y_val: 0


In [335]:
# Crea e allena il modello
model = BayesianRidge()
model.fit(X_train, y_train)

# Valutazione sul validation set
val_pred = model.predict(X_val)
rmse_val = np.sqrt(mean_squared_error(y_val, val_pred))
print(f"Validation RMSE: {rmse_val:.6f}")

Validation RMSE: 0.010272


In [336]:
# === Volatility Scaling Calibration (BayesianRidge version) ===
def strategy_stats(returns, exposures):
    """Compute Sharpe and volatility for a given exposure series."""
    strat = exposures * returns
    mean = np.nanmean(strat)
    std  = np.nanstd(strat)
    sharpe = (mean / (std + 1e-9)) * np.sqrt(252)
    vol = std * np.sqrt(252)
    return {'sharpe': sharpe, 'vol': vol}

# Ensure validation data has no NaN values
val_features_clean = val_p[final_features].fillna(0)

# Predict mean and std (Bayesian posterior uncertainty)
val_pred_mean, val_pred_std = model.predict(val_features_clean, return_std=True)

# Define market volatility for scaling reference
market_vol = np.nanstd(train_p['forward_returns']) * np.sqrt(252)
print(f"Market annualized volatility: {market_vol:.4f}")

# We'll use a dynamic confidence weight: confidence = 1 / (1 + std)
confidence = 1 / (1 + val_pred_std)
val_conf_adj = val_pred_mean * confidence  # lower exposure when uncertainty is high

# Grid search for best scaling factor k (Sharpe ratio under vol constraint)
best_k, best_sharpe = 0.1, -1e9
for k in np.linspace(0.01, 5.0, 100):
    exposures = np.clip((k * val_conf_adj), 0, 2)
    stats = strategy_stats(val_p['forward_returns'], exposures)
    if stats['vol'] <= VOL_MULTIPLIER_LIMIT * market_vol and stats['sharpe'] > best_sharpe:
        best_k = k
        best_sharpe = stats['sharpe']

print(f"Chosen scaling factor k={best_k:.3f} | Validation Sharpe={best_sharpe:.2f}")

Market annualized volatility: 0.1698
Chosen scaling factor k=5.000 | Validation Sharpe=0.69


In [337]:
## Test Predictions + Smoothing
test_features_clean = test_p[final_features].fillna(0)
test_pred = model.predict(test_features_clean)

alpha = 0.8
smoothed_allocation = []
prev = 0.0
for x in np.clip(best_k * test_pred, 0, 2):
    s = alpha * x + (1 - alpha) * prev
    smoothed_allocation.append(s)
    prev = s
smoothed_allocation = np.array(smoothed_allocation)

# replace in final submission
submission_df = pd.DataFrame({
    'date_id': test_p['date_id'],
    'prediction': smoothed_allocation  
})
# submission_df.to_csv("submission_br.csv", index=False)
print("Saved submission_br.csv")

Saved submission_br.csv


In [338]:
# Install shap if not already installed: pip install shap
import shap

print("Creating SHAP explainer for Bayesian Ridge model...")

# For linear models like BayesianRidge, we can use the Linear explainer
explainer = shap.LinearExplainer(model, X_train)
shap_values = explainer.shap_values(X_train)

# # Alternative: Use the general Explainer (slower but works for any model)
# explainer = shap.Explainer(model, X_train)
# shap_values = explainer(X_train)

# Get feature importance as mean absolute SHAP values
feature_importance_shap = pd.DataFrame({
'feature': X_train.columns,
'importance': np.abs(shap_values).mean(axis=0)
}).sort_values('importance', ascending=False)

print("\nTop 20 Most Important Features (SHAP values):")
print(feature_importance_shap.head(20))

# Plot SHAP summary
# shap.summary_plot(shap_values, X_train, show=False)

Creating SHAP explainer for Bayesian Ridge model...

Top 20 Most Important Features (SHAP values):
    feature  importance
100      m4    0.000545
170     v13    0.000512
80      m11    0.000471
146      s2    0.000410
54       e8    0.000371
110      m9    0.000344
44       e3    0.000243
132      p7    0.000242
6        d4    0.000233
82      m12    0.000209
40       e2    0.000209
138      s1    0.000203
116     p11    0.000197
96       m2    0.000192
130      p6    0.000189
134      p8    0.000161
60       i2    0.000149
118     p12    0.000139
98       m3    0.000126
144     s12    0.000121


In [339]:
"""
Kaggle Evaluation Metric:

strategy_returns = risk_free_rate * (1 - position) + position * forward_returns

if position = 0 → invest in risk-free asset,

if position = 1 → invest like the market,

if position = 2 → you are leveraged ×2 on the market.


def score():

strategy_returns = rf * (1 - pos) + pos * fwd_returns

In the code, the calibration seeks the best Sharpe of the portfolio exposed to pos by calculating:

strat = exposures * returns
""";

In [340]:
# # === Kaggle Inference Server Wrapper (BayesianRidge version) ===

# _model = model                     # BayesianRidge fitted model
# _best_k = best_k                   # scaling factor from validation calibration
# _history_returns = list(train_p['forward_returns'].iloc[-VOL_WINDOW:].tolist())

# def predict(pl_df: pl.DataFrame) -> float:
#     """Predict allocation for one timestep (Kaggle competition API)."""
#     global _history_returns
#     # Convert Polars → Pandas
#     pdf = pl_df.to_pandas()
    
#     # Apply same preprocessing
#     pdf_p = prepare_df(pdf, median_map, feature_cols)
#     for f in final_features:
#         if f not in pdf_p.columns:
#             pdf_p[f] = 0.0
    
#     # Convert to NumPy and predict
#     x = pdf_p[final_features].to_numpy()

#     """
#     added standard deviation based confidence adjustment
#     to reduce allocation when uncertainty is high
#     """

#     pred, std = model.predict(x, return_std=True)
    
#     # Compute rolling volatility estimate
#     vol_est = np.std(_history_returns) or 1e-3
#     confidence = 1 / (1 + std)
#     alloc = float(np.clip(_best_k * pred * confidence / (vol_est + 1e-9), 0, 2))
    
#     # Update history (for volatility tracking)
#     if 'lagged_forward_returns' in pl_df.columns:
#         try:
#             _history_returns.append(float(pl_df['lagged_forward_returns'][0]))
#         except:
#             _history_returns.append(0.0)
#     else:
#         _history_returns.append(0.0)
    
#     # Keep only the last VOL_WINDOW entries
#     _history_returns = _history_returns[-VOL_WINDOW:]
#     return alloc

# # Instantiate the Kaggle inference server
# server = kdeval.DefaultInferenceServer(predict)

# if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
#     server.serve()
# else:
#     server.run_local_gateway((str(DATA_DIR),))
